# Assignment 03 - Basic RAG Architecture



## Load Environment Variables
All the environment variables, APIS keys etc are loaded in the `.env` file. Load it into memory.

In [3]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

print("ℹ️  Loaded environment variables from .env file")

ℹ️  Loaded environment variables from .env file


## Define Data Sources

Real Meta PDFs from their Investor Relations website

In [4]:
# Real Meta earnings data sources
data_sources = {
    "Q1": {
        "press_release": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q1/Meta-03-31-2025-Exhibit-99-1-Final.pdf",
        "presentation": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q1/Earnings-Presentation-Q1-2025-FINAL.pdf",
        "earnings_call": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q1/Transcripts/META-Q1-2025-Earnings-Call-Transcript-1.pdf",
        "followup_qa": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q1/Transcripts/META-Q1-2025-Follow-Up-Call-Transcript.pdf"
    },
    "Q2": {
        "press_release": "https://s21.q4cdn.com/399680738/files/doc_downloads/Meta-06-30-2025-Exhibit-99-1-FINAL.pdf",
        "presentation": "https://s21.q4cdn.com/399680738/files/doc_downloads/Earnings-Presentation-Q2-2025-FINAL.pdf",
        "earnings_call": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q2/META-Q2-2025-Earnings-Call-Transcript.pdf",
        "followup_qa": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q2/META-Q2-2025-Follow-Up-Call-Transcript.pdf"
    },
    "Q3": {
        "press_release": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q3/Meta-09-30-2025-Exhibit-99-1-Final.pdf",
        "presentation": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q3/Earnings-Presentation-Q3-2025-Final.pdf",
        "earnings_call": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q3/META-Q3-2025-Earnings-Call-Transcript.pdf",
        "followup_qa": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q3/META-Q3-2025-Follow-Up-Call-Transcript.pdf"
    },
    "Q4": {
        "press_release": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q4/Meta-12-31-2025-Exhibit-99-1-FINAL.pdf",
        "presentation": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q4/Earnings-Presentation-Q4-2025-FINAL.pdf",
        "earnings_call": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q4/META-Q4-2025-Earnings-Call-Transcript.pdf",
        "followup_qa": "https://s21.q4cdn.com/399680738/files/doc_financials/2025/q4/META-Q4-2025-Follow-Up-Call-Transcript.pdf"
    }
}

print("✅ Data sources loaded!")
print()
print("Available quarters: Q1, Q2, Q3, Q4")
print("Document types per quarter: press_release, presentation, earnings_call, followup_qa")
print()
print(f"Total documents to process: {len(data_sources) * 4} PDFs")

✅ Data sources loaded!

Available quarters: Q1, Q2, Q3, Q4
Document types per quarter: press_release, presentation, earnings_call, followup_qa

Total documents to process: 16 PDFs


## Initialize Wrappers

Initialize the LLM Wrappers / Agents

In [5]:
from openai import OpenAI
import chromadb

# Initialize OpenAI client
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))

# Initialize ChromaDB client (in-memory database, no API keys needed)
chroma_client = chromadb.Client()

## Utility Functions

Helper functions for PDF fetching and chunk storage

In [6]:
# Import required libraries
import requests
from io import BytesIO
from PyPDF2 import PdfReader
import fitz  # PyMuPDF

def fetch_and_extract_pdf(pdf_url, timeout=10, return_type="Text"):
    """
    Fetch a PDF from a URL and extract all text
    
    Args:
        pdf_url: URL to the PDF file
        timeout: Request timeout in seconds
        return_pages: If True, return list of pages; if False, return combined text
    
    Returns:
        If return_pages=False: Extracted text (string) or None if error
        If return_pages=True: List of page texts or None if error
    """
    try:
        response = requests.get(pdf_url, timeout=timeout)
        response.raise_for_status()
        
        pdf_file = BytesIO(response.content)
        reader = PdfReader(pdf_file)
        
        if return_type == "Pages":
            pages = []
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text and page_text.strip():
                    pages.append(page_text.strip())
            return pages if pages else None
        elif return_type == "Paragraphs":
            paragraphs = []
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    for para in page_text.split("\n\n"):
                        if para.strip():
                            paragraphs.append(para.strip())
            return paragraphs if paragraphs else None
        else: # Text
            text = ""
            for page in reader.pages:
                page_text = page.extract_text()
                if page_text:
                    text += page_text + "\n"
            
            return text.strip()
    
    except Exception as e:
        print(f"   ⚠️  Error fetching PDF: {str(e)[:60]}")
        return None

def fetch_and_extract_section_from_pdf(pdf_url, timeout=10):
    """
    Fetch a PDF from a URL and extract a specific section using PyMuPDF
    
    Args:
        pdf_url: URL to the PDF file
        timeout: Request timeout in seconds
    
    Returns:
        Extracted section text (string) or None if error
    """
    try:
        doc = fitz.open(stream=requests.get(pdf_url, timeout=timeout).content, filetype="pdf")
        sections = []
        current_section = None
        
        for page_num, page in enumerate(doc):
             blocks = page.get_text("dict")["blocks"]
             
             for block in blocks:
                if "lines" not in block:
                    continue
                
                for line in block["lines"]:
                    line_text = ""
                    max_font_size = 0
                    is_bold = False
                    
                    for span in line["spans"]:
                        for span in line["spans"]:
                            text = span["text"].strip()
                            if not text:
                                continue
                            
                            line_text += text + " "
                            font_size = span["size"]
                            max_font_size = max(max_font_size, font_size)
                            
                            if "bold" in span["font"].lower():
                                is_bold = True
                        line_text = line_text.strip()
                        
                    # Detect section headings (adjust threshold based on your PDF)
                    if max_font_size > 11 or is_bold or line_text.isupper():
                        # Check if it's actually a heading (not too long)
                        if len(line_text) < 100 and line_text:
                            # Save previous section
                            if current_section and current_section["content"].strip():
                                sections.append(current_section)
                            
                            # Start new section
                            current_section = {
                                "title": line_text,
                                "content": "",
                                "page_start": page_num + 1,
                                "font_size": max_font_size
                            }
                    elif current_section:
                        current_section["content"] += line_text + "\n"   
                        
        # Add last section
        if current_section and current_section["content"].strip():
            sections.append(current_section)
            
        return sections if sections else None
    
    except Exception as e:
        print(f"   ⚠️  Error fetching PDF section: {str(e)[:60]}")
        return None

def add_chunk_to_collection(collection, doc_type, quarter, chunk_text, chunk_id):
    """
    Add a text chunk to a ChromaDB collection
    
    Args:
        collection: ChromaDB collection object
        doc_type: Type of document (e.g., 'earnings_call')
        quarter: Quarter (Q1, Q2, Q3)
        chunk_text: The text content of the chunk
        chunk_id: Unique identifier for the chunk
    """
    metadata = {
        "doc_type": doc_type,
        "quarter": quarter,
        "chunk_length": len(chunk_text)
    }
    
    collection.add(
        ids=[chunk_id],
        documents=[chunk_text],
        metadatas=[metadata]
    )

def query_collection(collection, doc_type=None, quarter=None):
    """
    Query a ChromaDB collection with optional filters
    
    Args:
        collection: ChromaDB collection object
        doc_type: Optional filter by document type
        quarter: Optional filter by quarter
    
    Returns:
        Query results dict with 'documents', 'metadatas', 'ids'
    """
    where_filter = None
    conditions = []
    
    if doc_type:
        conditions.append({"doc_type": {"$eq": doc_type}})
    if quarter:
        conditions.append({"quarter": {"$eq": quarter}})
    
    if conditions:
        if len(conditions) == 1:
            where_filter = conditions[0]
        else:
            where_filter = {"$and": conditions}
    
    return collection.get(where=where_filter)

def generate_rag_answer(query, chunks, max_chunks=5):
    """
    Generate an answer using GPT-4o based on a query and retrieved chunks
    
    Args:
        query: The user's question
        chunks: List of text chunks to use as context
        max_chunks: Maximum number of chunks to use
    
    Returns:
        Generated answer (string) or None if error
    """
    try:
        # Limit chunks to avoid token overflow
        chunks_to_use = chunks[:max_chunks]
        context = "\n\n".join([f"[Chunk {i+1}]\n{chunk}" for i, chunk in enumerate(chunks_to_use)])
        
        system_prompt = """You are a helpful financial analyst assistant. 
Based on the provided document chunks, answer the user's question accurately and concisely.
If the information is not in the provided chunks, say 'I don't have enough information to answer this question.'
Always cite which chunks you used to answer the question."""
        
        user_message = f"""Question: {query}

Document Chunks:
{context}

Please provide a clear, concise answer based on the above information."""
        
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_message}
            ],
            temperature=0.7,
            max_tokens=300
        )
        
        return response.choices[0].message.content
    
    except Exception as e:
        print(f"   ⚠️  Error generating answer: {str(e)[:60]}")
        return None

print("✅ Utility functions defined!")
print()
print("Functions available:")
print("  - fetch_and_extract_pdf(url): Fetch and extract text from PDFs")
print("  - add_chunk_to_collection(): Add chunks to a collection")
print("  - query_collection(): Query a collection")
print("  - generate_rag_answer(): Generate LLM answers from chunks using GPT-4o")

✅ Utility functions defined!

Functions available:
  - fetch_and_extract_pdf(url): Fetch and extract text from PDFs
  - add_chunk_to_collection(): Add chunks to a collection
  - query_collection(): Query a collection
  - generate_rag_answer(): Generate LLM answers from chunks using GPT-4o


## Initialize ChromaDB Collections

> The details below
- Strategy 3 - By Paragraphs
- Strategy 4 - By Fixed token length

### Strategy 1 - By Page

> Details below

#### Build Index

In [7]:
print("📊 Strategy 1 - ByPage: BUILDING INDEX: PAGE-BASED CHUNKING (Each Page = 1 Chunk)")
print("="*70)

s1_bypage_collection = chroma_client.get_or_create_collection(
    name="Strategy1_ByPage_Collection",
    metadata={"description": "Chunks created by treating each PDF page as a separate chunk"}
)

s1_chunk_count = 0
for quarter in data_sources:
    print(f"\n🔄 Processing {quarter}...")
    for doc_type in data_sources[quarter]:
        print(f"  📄 {doc_type}...", end="")
        
        # Fetch PDF data
        pdf_url = data_sources[quarter][doc_type]
        pages = fetch_and_extract_pdf(pdf_url, return_type="Pages")
        
        if not pages:
            print(f"   ⚠️  No text extracted from {pdf_url}")
            continue
        
        # Strategy 1: Each page is one chunk  
        for i, page_text in enumerate(pages):
            chunk_id = f"{quarter}_{doc_type}_s1_page_{i+1}"
            add_chunk_to_collection(s1_bypage_collection, doc_type, quarter, page_text, chunk_id)
            s1_chunk_count += 1
            
        print(f" ✅ {i} pages")
        
print(f"\n✅ Strategy 1: Added {s1_chunk_count} page-based chunks to collection!")

📊 Strategy 1 - ByPage: BUILDING INDEX: PAGE-BASED CHUNKING (Each Page = 1 Chunk)

🔄 Processing Q1...
  📄 press_release... ✅ 8 pages
  📄 presentation... ✅ 17 pages
  📄 earnings_call... ✅ 18 pages
  📄 followup_qa... ✅ 10 pages

🔄 Processing Q2...
  📄 press_release... ✅ 8 pages
  📄 presentation... ✅ 17 pages
  📄 earnings_call... ✅ 18 pages
  📄 followup_qa... ✅ 9 pages

🔄 Processing Q3...
  📄 press_release... ✅ 8 pages
  📄 presentation... ✅ 17 pages
  📄 earnings_call... ✅ 18 pages
  📄 followup_qa... ✅ 9 pages

🔄 Processing Q4...
  📄 press_release... ✅ 8 pages
  📄 presentation... ✅ 17 pages
  📄 earnings_call... ✅ 20 pages
  📄 followup_qa... ✅ 10 pages

✅ Strategy 1: Added 228 page-based chunks to collection!


#### Inspect Index

In [8]:
print("📊 Strategy 1 - ByPage: INSPECT INDEX : PAGE-BASED CHUNKING (Each Page = 1 Chunk)")
print("="*70)

# Get all documents in the collection
s1_all_docs = query_collection(s1_bypage_collection)

print(f"\n✅ Total documents in ByPage Index: {len(s1_all_docs['documents'])}")
print(f"Strategy 1 - ByPage - Statistics:")
s1_chunk_sizes = [m['chunk_length'] for m in s1_all_docs['metadatas']]
print(f"   Average chunk size: {sum(s1_chunk_sizes) / len(s1_chunk_sizes):.0f} characters")
print(f"   Minimum chunk size: {min(s1_chunk_sizes)} characters")
print(f"   Maximum chunk size: {max(s1_chunk_sizes)} characters")

📊 Strategy 1 - ByPage: INSPECT INDEX : PAGE-BASED CHUNKING (Each Page = 1 Chunk)

✅ Total documents in ByPage Index: 228
Strategy 1 - ByPage - Statistics:
   Average chunk size: 2175 characters
   Minimum chunk size: 8 characters
   Maximum chunk size: 5375 characters


### Strategy 2 - By Sentences (3)

> Details below

#### Build Index

In [14]:
print("📊 Strategy 2 - BySentences: BUILDING INDEX: SENTENCE BASED CHUNKING (3 x Sentences = 1 Chunk)")
print("="*70)

s2_bysentences_collection = chroma_client.get_or_create_collection(
    name="Strategy2_BySentences_Collection",
    metadata={"description": "Chunks created by grouping every 3 sentences together"}
)

s2_chunk_count = 0
for quarter in data_sources:
    print(f"\n🔄 Processing {quarter}...")
    for doc_type in data_sources[quarter]:
        print(f"  📄 {doc_type}...", end="")
        
        # Fetch PDF data
        pdf_url = data_sources[quarter][doc_type]
        text = fetch_and_extract_pdf(pdf_url, return_type="Text")
        
        if not text:
            print(f"   ⚠️  No text extracted from {pdf_url}")
            continue
        
        # Strategy 2: Group every 3 sentences into one chunk
        sentences = [s.strip() for s in text.split('.') if s.strip()]
        for i in range(0, len(sentences), 3):
            chunk_sentences = sentences[i:i+3]
            chunk_text = '. '.join(chunk_sentences) + ('.' if chunk_sentences else '')
            chunk_id = f"{quarter}_{doc_type}_s2_sentences_{i//3 + 1}"
            add_chunk_to_collection(s2_bysentences_collection, doc_type, quarter, chunk_text, chunk_id)
            s2_chunk_count += 1
            
        print(f" ✅ {len(sentences)} sentences grouped into {len(sentences) // 3 + (1 if len(sentences) % 3 > 0 else 0)} chunks")
        
print(f"\n✅ Strategy 2: Added {s2_chunk_count} sentence-based chunks to collection!")

📊 Strategy 2 - BySentences: BUILDING INDEX: SENTENCE BASED CHUNKING (3 x Sentences = 1 Chunk)

🔄 Processing Q1...
  📄 press_release... ✅ 108 sentences grouped into 36 chunks
  📄 presentation... ✅ 99 sentences grouped into 33 chunks
  📄 earnings_call... ✅ 460 sentences grouped into 154 chunks
  📄 followup_qa... ✅ 275 sentences grouped into 92 chunks

🔄 Processing Q2...
  📄 press_release... ✅ 120 sentences grouped into 40 chunks
  📄 presentation... ✅ 99 sentences grouped into 33 chunks
  📄 earnings_call... ✅ 459 sentences grouped into 153 chunks
  📄 followup_qa... ✅ 234 sentences grouped into 78 chunks

🔄 Processing Q3...
  📄 press_release... ✅ 144 sentences grouped into 48 chunks
  📄 presentation... ✅ 114 sentences grouped into 38 chunks
  📄 earnings_call... ✅ 443 sentences grouped into 148 chunks
  📄 followup_qa... ✅ 243 sentences grouped into 81 chunks

🔄 Processing Q4...
  📄 press_release... ✅ 128 sentences grouped into 43 chunks
  📄 presentation... ✅ 115 sentences grouped into 39 ch

#### Inspect Index

In [16]:
print("📊 Strategy 2 - BySentence: INSPECT INDEX : SENTENCE BASED CHUNKING (3 x Sentences = 1 Chunk)")
print("="*70)

# Get all documents in the collection
s2_all_docs = query_collection(s2_bysentences_collection)

print(f"\n✅ Total documents in BySentence Index: {len(s2_all_docs['documents'])}")
print(f"Strategy 2 - BySentence - Statistics:")
s2_chunk_sizes = [m['chunk_length'] for m in s2_all_docs['metadatas']]
print(f"   Average chunk size: {sum(s2_chunk_sizes) / len(s2_chunk_sizes):.0f} characters")
print(f"   Minimum chunk size: {min(s2_chunk_sizes)} characters")
print(f"   Maximum chunk size: {max(s2_chunk_sizes)} characters")

📊 Strategy 2 - BySentence: INSPECT INDEX : SENTENCE BASED CHUNKING (3 x Sentences = 1 Chunk)

✅ Total documents in BySentence Index: 1261
Strategy 2 - BySentence - Statistics:
   Average chunk size: 389 characters
   Minimum chunk size: 4 characters
   Maximum chunk size: 4926 characters


### Strategy 3 - By Sections

> Details below

#### Build Index

In [10]:
print("📊 Strategy 3 - BySections: BUILDING INDEX: SECTION BASED CHUNKING (1 Section = 1 Chunk)")
print("="*70)

s3_bysections_collection = chroma_client.get_or_create_collection(
    name="Strategy3_BySections_Collection",
    metadata={"description": "Chunks created by treating each section as a separate chunk"}
)

s3_chunk_count = 0
for quarter in data_sources:
    print(f"\n🔄 Processing {quarter}...")
    for doc_type in data_sources[quarter]:
        print(f"  📄 {doc_type}...", end="")
        
        # Fetch PDF data
        pdf_url = data_sources[quarter][doc_type]
        # paragraphs = fetch_and_extract_pdf(pdf_url, return_type="Paragraphs")
        
        # if not paragraphs:
        #     print(f"   ⚠️  No text extracted from {pdf_url}")
        #     continue
        
        # print(f"   Extracted {len(paragraphs)} paragraphs from PDF", end="")
        # # Strategy 3: Each paragraph is one chunk  
        # for i, para_text in enumerate(paragraphs):
        #     print(".", end="")  # Progress indicator for large number of paragraphs
        #     chunk_id = f"{quarter}_{doc_type}_s3_para_{i+1}"
        #     add_chunk_to_collection(s3_byparagraphs_collection, doc_type, quarter, para_text, chunk_id)
        #     s3_chunk_count += 1
            
        # print(f" ✅ {len(paragraphs)} paragraphs")
        sections = fetch_and_extract_section_from_pdf(pdf_url)
        if not sections or len(sections) == 0:
            print(f"   ⚠️  No sections extracted from {pdf_url}")
            print(f"   Falling back to Sentence-based chunking...")
            pages = fetch_and_extract_pdf(pdf_url, return_type="Pages")
            if not pages:
                print(f"   ⚠️  No text extracted from {pdf_url}")
                continue
            for i, page_text in enumerate(pages):
                chunk_id = f"{quarter}_{doc_type}_s1_page_{i+1}"
                add_chunk_to_collection(s3_bysections_collection, doc_type, quarter, page_text, chunk_id)
                s3_chunk_count += 1
            # Add pages to collection as fallback
            continue
        
        print(f" ✅ Extracted {len(sections) if sections else 0} sections")
        for section in sections:
                chunk_id = f"{quarter}_{doc_type}_s3_section_{s3_chunk_count + 1}"
                add_chunk_to_collection(s3_bysections_collection, doc_type, quarter, section['content'], chunk_id)
                s3_chunk_count += 1
            
print(f"\n✅ Strategy 3: Added {s3_chunk_count} section-based chunks to collection!")

📊 Strategy 3 - BySections: BUILDING INDEX: SECTION BASED CHUNKING (1 Section = 1 Chunk)

🔄 Processing Q1...
  📄 press_release... ✅ Extracted 44 sections
  📄 presentation...   ⚠️  No sections extracted from https://s21.q4cdn.com/399680738/files/doc_financials/2025/q1/Earnings-Presentation-Q1-2025-FINAL.pdf
   Falling back to Sentence-based chunking...
  📄 earnings_call...   ⚠️  No sections extracted from https://s21.q4cdn.com/399680738/files/doc_financials/2025/q1/Transcripts/META-Q1-2025-Earnings-Call-Transcript-1.pdf
   Falling back to Sentence-based chunking...
  📄 followup_qa... ✅ Extracted 11 sections

🔄 Processing Q2...
  📄 press_release... ✅ Extracted 43 sections
  📄 presentation...   ⚠️  No sections extracted from https://s21.q4cdn.com/399680738/files/doc_downloads/Earnings-Presentation-Q2-2025-FINAL.pdf
   Falling back to Sentence-based chunking...
  📄 earnings_call...   ⚠️  No sections extracted from https://s21.q4cdn.com/399680738/files/doc_financials/2025/q2/META-Q2-2025-Ear

#### Inspect Index

In [11]:
print("📊 Strategy 3 - ByParagraphs: INSPECT INDEX : PARAGRAPH BASED CHUNKING (1 Paragraph = 1 Chunk)")
print("="*70)

# Get all documents in the collection
s3_all_docs = query_collection(s3_bysections_collection)
print(f"\n✅ Total documents in BySection Index: {len(s3_all_docs['documents'])}")
print(f"Strategy 3 - BySections - Statistics:")
s3_chunk_sizes = [m['chunk_length'] for m in s3_all_docs['metadatas']]
print(f"   Average chunk size: {sum(s3_chunk_sizes) / len(s3_chunk_sizes):.0f} characters")
print(f"   Minimum chunk size: {min(s3_chunk_sizes)} characters")
print(f"   Maximum chunk size: {max(s3_chunk_sizes)} characters")

📊 Strategy 3 - ByParagraphs: INSPECT INDEX : PARAGRAPH BASED CHUNKING (1 Paragraph = 1 Chunk)

✅ Total documents in BySection Index: 356
Strategy 3 - BySections - Statistics:
   Average chunk size: 1299 characters
   Minimum chunk size: 6 characters
   Maximum chunk size: 38090 characters


### Strategy 4 - Fixed Token Length

#### Build Index

In [18]:
print("📊 Strategy 4 - Fixed Token: BUILDING INDEX: FIXED TOKEN CHUNKING (1 Chunk = Fixed Token Length)")
print("="*70)

s4_fixedtoken_collection = chroma_client.get_or_create_collection(
    name="Strategy4_FixedToken_Collection",
    metadata={"description": "Chunks created by splitting text into fixed token lengths (e.g., 1000 tokens per chunk)"}
)

s4_chunk_count = 0
for quarter in data_sources:
    print(f"\n🔄 Processing {quarter}...")
    for doc_type in data_sources[quarter]:
        print(f"  📄 {doc_type}...", end="")
        
        # Fetch PDF data
        pdf_url = data_sources[quarter][doc_type]
        text = fetch_and_extract_pdf(pdf_url, return_type="Text")
        
        if not text:
            print(f"   ⚠️  No text extracted from {pdf_url}")
            continue
        
        # Strategy 4: Split text into fixed token length chunks (e.g., 1000 tokens)
        # For simplicity, we'll approximate tokens by splitting on whitespace and punctuation
        chunk_size = 1000  # Approximate number of tokens per chunk
        overlap = 200    # Number of words to overlap between chunks to maintain context
        
        start = 0
        while start < len(text):
             end = start + chunk_size
             chunk_text = text[start:end]
             
             chunk_id = f"{quarter}_{doc_type}_s4_fixedtoken_{s4_chunk_count + 1}"
             add_chunk_to_collection(s4_fixedtoken_collection, doc_type, quarter, chunk_text, chunk_id)
             s4_chunk_count += 1
             start += (chunk_size - overlap)  # Move start forward with overlap
            
        print(f" ✅ {len(text.split())} words split into {len(text.split()) // chunk_size + (1 if len(text.split()) % chunk_size > 0 else 0)} chunks")

📊 Strategy 4 - Fixed Token: BUILDING INDEX: FIXED TOKEN CHUNKING (1 Chunk = Fixed Token Length)

🔄 Processing Q1...
  📄 press_release... ✅ 2519 words split into 3 chunks
  📄 presentation... ✅ 2676 words split into 3 chunks
  📄 earnings_call... ✅ 9344 words split into 10 chunks
  📄 followup_qa... ✅ 5372 words split into 6 chunks

🔄 Processing Q2...
  📄 press_release... ✅ 2974 words split into 3 chunks
  📄 presentation... ✅ 2668 words split into 3 chunks
  📄 earnings_call... ✅ 9622 words split into 10 chunks
  📄 followup_qa... ✅ 4607 words split into 5 chunks

🔄 Processing Q3...
  📄 press_release... ✅ 3277 words split into 4 chunks
  📄 presentation... ✅ 2861 words split into 3 chunks
  📄 earnings_call... ✅ 9682 words split into 10 chunks
  📄 followup_qa... ✅ 4645 words split into 5 chunks

🔄 Processing Q4...
  📄 press_release... ✅ 3019 words split into 4 chunks
  📄 presentation... ✅ 2925 words split into 3 chunks
  📄 earnings_call... ✅ 9995 words split into 10 chunks
  📄 followup_qa... ✅

#### Inspect Index

In [20]:
print("📊 Strategy 4 - Fixed Token: INSPECT INDEX: FIXED TOKEN CHUNKING (1 Chunk = Fixed Token Length)")
print("="*70)

# Get all documents in the collection
s4_all_docs = query_collection(s4_fixedtoken_collection)
print(f"\n✅ Total documents in Fixed Token Index: {len(s4_all_docs['documents'])}")
print(f"Strategy 4 - Fixed Token - Statistics:")
s4_chunk_sizes = [m['chunk_length'] for m in s4_all_docs['metadatas']]
print(f"   Average chunk size: {sum(s4_chunk_sizes) / len(s4_chunk_sizes):.0f} characters")
print(f"   Minimum chunk size: {min(s4_chunk_sizes)} characters")
print(f"   Maximum chunk size: {max(s4_chunk_sizes)} characters")

📊 Strategy 4 - Fixed Token: INSPECT INDEX: FIXED TOKEN CHUNKING (1 Chunk = Fixed Token Length)

✅ Total documents in Fixed Token Index: 630
Strategy 4 - Fixed Token - Statistics:
   Average chunk size: 982 characters
   Minimum chunk size: 4 characters
   Maximum chunk size: 1000 characters


## Index Comparison

In [21]:
print(f"\n📊 INDEX COMPARISON:")
print("-" * 70)

print(f"Strategy 1 - ByPage     : Documents = {len(s1_all_docs['documents']):5d} | Avg Size = {sum([m['chunk_length'] for m in s1_all_docs['metadatas']]) / len(s1_all_docs['metadatas']):6.0f} chars")
print(f"Strategy 2 - BySentences: Documents = {len(s2_all_docs['documents']):5d} | Avg Size = {sum([m['chunk_length'] for m in s2_all_docs['metadatas']]) / len(s2_all_docs['metadatas']):6.0f} chars")
print(f"Strategy 3 - BySections : Documents = {len(s3_all_docs['documents']):5d} | Avg Size = {sum([m['chunk_length'] for m in s3_all_docs['metadatas']]) / len(s3_all_docs['metadatas']):6.0f} chars")
print(f"Strategy 4 - FixedToken : Documents = {len(s4_all_docs['documents']):5d} | Avg Size = {sum([m['chunk_length'] for m in s4_all_docs['metadatas']]) / len(s4_all_docs['metadatas']):6.0f} chars")


📊 INDEX COMPARISON:
----------------------------------------------------------------------
Strategy 1 - ByPage     : Documents =   228 | Avg Size =   2175 chars
Strategy 2 - BySentences: Documents =  1261 | Avg Size =    389 chars
Strategy 3 - BySections : Documents =   356 | Avg Size =   1299 chars
Strategy 4 - FixedToken : Documents =   630 | Avg Size =    982 chars
